In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# load data
x_train = pd.read_csv('training_set.csv', header=None, names=['x1', 'x2', 'target'])
x_val = pd.read_csv('validation_set.csv', header=None, names=['x1', 'x2', 'target'])

x_train.plot.scatter('x1', 'x2', c='target')
x_val.plot.scatter('x1', 'x2', c='target')

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Preprocessing

x_train = pd.read_csv('training_set.csv', header=None, names=['x1', 'x2', 'target'])
x_val = pd.read_csv('validation_set.csv', header=None, names=['x1', 'x2', 'target'])

x_train['x1'] = (x_train['x1'] - x_train['x1'].mean())/x_train['x1'].std() # normalise to ~N(0,1)
x_train['x2'] = (x_train['x2'] - x_train['x2'].mean())/x_train['x2'].std() # normalise to ~N(0,1)

x_val['x1'] = (x_val['x1'] - x_val['x1'].mean())/x_val['x1'].std()
x_val['x2'] = (x_val['x2'] - x_val['x2'].mean())/x_val['x2'].std()

x_train.plot.scatter('x1', 'x2', c='target')
x_val.plot.scatter('x1', 'x2', c='target')

NameError: name 'pd' is not defined

In [3]:
# Helper function
def g_prime(a: float) -> float:
    return 1/(np.cosh(a)**2)


# Network class
class Network:
    def __init__(self, M):
        self.M = M # Number of neurons
        self.w = np.random.normal(0,1,(M, 2)) # Hidden layer
        self.th = np.random.normal(0,1,(M, )) # Hidden layer
        self.W = np.random.normal(0,1,(M,  )) # Output layer
        self.T = np.random.normal(0,1)        # Output layer
        self.lr = 0.001                       # learning rate
        self.pval = 5000
        self.ptrain = 10000
        self.iteration = 0
        self.c_errors = []
        
        
    def feed_forward(self, x : list) -> float: 
        # Hidden layer
        b = self.w @ x - self.th
        V = np.tanh(b)
        # Output layer
        B = np.dot(self.W, V) - self.T
        O = np.tanh(B)
        
        return b, V, B, O
    
    
    def train_stochastic(self, x_train : pd.DataFrame, x_val : pd.DataFrame):
        
        # Choose 1 random patterns
        # feed
        # backpropagate
        # validate every 10 patterns
        # repeat
        run = 1
        while run: 
            for _ in range(self.ptrain):
                idx = np.random.choice(self.ptrain)
                # Minibatching for speed
                row = x_train.iloc[idx]

                x = [row['x1'], row['x2']]
                t = row['target']

                b, V, B, O = self.feed_forward(x)

                # backpropagate
                g = (t - O) * g_prime(np.dot(self.W, V) - self.T)
                d = g * self.W * g_prime(self.w @ x - self.th)

                dW = g * V
                dT = g

                dw = np.outer(d, x)
                dth = d 

                self.W += self.lr * dW
                self.T -= self.lr * dT
                self.w += self.lr * dw
                self.th -= self.lr * dth

            # Validate
            c_error = self.validate(x_val)
            self.c_errors.append(c_error)
            
            if c_error > 0.118:
                print(f'iter: {self.iteration: >3}\tc_error {c_error}')
                self.iteration += 1
                # If 5 latest c_errors are identical and we have iterated ALOT
                if self.iteration > 2000:
                    self.iteration = 0
                    run = 0
            else:
                print(f'Found c_error lower than 0.12: c_error = {c_error}, iter: {self.iteration}')
                run = 0
                
                
    def classification_error(self, targets: list, predictions: list) -> float:
        
        c_error = 0
        
        O_sgn = np.sign(predictions)
        O_sgn[O_sgn == 0] = 1
        
        e_abs = np.abs(O_sgn - targets)
        c_error = np.sum(e_abs)/(2*self.pval)
        
        
        return c_error
    
    
    def validate(self, x_val):
        
        targets = list(x_val['target'])
        preds = []
        
        for index, row in x_val.iterrows():
            x_i = [row['x1'], row['x2']]
            preds.append(self.feed_forward(x_i)[-1])
        
        c_error = self.classification_error(list(x_val['target']), preds)
        
        return c_error
    
    


NameError: name 'pd' is not defined

In [ ]:
n = Network(20)
n.train_stochastic(x_train, x_val)

In [73]:
w_df = pd.DataFrame(n.w)
w_df.to_csv('w1.csv', header=False, index=False)
W_df = pd.DataFrame(n.W)
W_df.to_csv('w2.csv', header=False, index=False)
th_df = pd.DataFrame(n.th)
T_df = pd.DataFrame([n.T])
th_df.to_csv('t1.csv', header=False, index=False)

T_df.to_csv('t2.csv', header=False, index=False)